# Modeltraining

This notebook is used to train models that predict different stainings of microscopy images. 

In [ ]:
import os
import os.path
import numpy as np
import glob
import matplotlib.pyplot as plt
import preprocessing_functions as pre
import training_functions as training
import skimage.io
import prediction as pred
import keras

In [ ]:
# directories
dir_valim = r"C:\Users\tsche\Desktop\Master\2. Semester\Project Staining\validation"
dir_im = r"C:\Users\tsche\Desktop\Master\2. Semester\Project Staining\16bitImages"
dir_results = r'C:\Users\tsche\Desktop\Master\2. Semester\Project Staining\gitrepo\TeresaScheidt\training_results'

In [ ]:
# define image folders
val_images = glob.glob(dir_valim, '*d0.png') 
X_val = pre.stack_images(val_images, num_channels = 1)
val_images1 = glob.glob(dir_valim, '*d1.png')
Y_val_1 = pre.stack_images(val_images1, num_channels = 1)
val_images2 = glob.glob(dir_valim, '*d2.png')
Y_val_2 = pre.stack_images(val_images2, num_channels = 1)

In [ ]:
# Imageset
images1 = glob.glob(dir_im, '*d0.png')
val_images = [i.replace('validation','16bitImages') for i in val_images]
images =  [x for x in images1 if x not in val_images]
X_1 = pre.stack_images(images, num_channels = 1)
images2 = glob.glob(dir_valim, '*d0.png')
val_images2 = [i.replace('validation','16bitImages') for i in val_images2]
all_im = images1 + val_images2
images =  [x for x in images2 if x not in all_im]
Y_1 = pre.stack_images(images, num_channels = 2)

In [ ]:
# normalize images
(x_1, mean, std) = pre.norm_batch(X_1)
x_val = pre.norm(X_val, mean, std)
(y_1,mean,std) = pre.norm_batch(Y_1)
y_val = pre.norm(Y_val, mean, std)

In [ ]:
# recenter images
X_train = pre.recenter(x_1)
Y_train = pre.recenter(y)
x_val = pre.recenter(x_val)
y_val = pre.recenter(y_val)

In [ ]:
# define modelname
names = ['d1_mse_ssim_recentered','d2_mse_ssim_recentered']

In [ ]:
import importlib
import cv2
importlib.reload(training)
for name in names:
    # define imageset used for each model here
    
    #if name == 'd1_mse_ssim_recentered':
     #   Y_train = Y_train_1
     #  y_val = y_val_1
    #if name == 'd2_mse_ssim_recentered':
     #   Y_train = Y_train_2
     #   y_val = y_val_2
    
    history = training.train(X_train,Y_train, x_val, y_val,2, name)
    dir = os.path.join(dir_results, name +'_loss.png')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(name)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(dir)
    plt.show()
    model = keras.models.load_model(name, compile=False)
    image = x_val[0]
    output = pred.predict(model, image, 128, 128, num_outputs=2)
    output = pre.center_back(output)
    output_norm = pre.unnormalize(output, mean, std)
    im = output_norm[:,:,0]
    dir_d1 = os.path.join(dir_results, name + '_d1.png')
    dir_d2 = os.path.join(dir_results, name +'_d2.png')
    cv2.imwrite(dir_d1, im.astype(np.uint16))
    #skimage.io.imsave(dir_d1, im) #for 8bit image
    im = output_norm[:,:,1]
    cv2.imwrite(dir_d2, im.astype(np.uint16))
    #skimage.io.imsave(dir_d2, im)